# Econ 1193 - Project 3: Demand Estimation 

Group Members: Behera Bishwa, Gurreonero Akaya, Seegars Chelsea

### 1. Import Required Packages

In [307]:
import pandas as pd  # for reading data, manipulating DataFrames, and summarizing data
import numpy as np   # for numerical operations
import statsmodels.formula.api as smf  # for formula-style regression
import math  # for category binning

### 2. Load Data

We load the 2005 car data from GitHub and report the shape of the dataset

In [310]:
# Load car dataset
car_url = "https://raw.githubusercontent.com/ArieBeresteanu/Econ1193_Spring2025/main/data/cardata2005.json"
cars = pd.read_json(car_url)
print("Number of cars:", cars.shape[0])
print("Number of columns:", cars.shape[1])

Number of cars: 217
Number of columns: 20


### 3. Add Car Category Names

We assign each car to a broad category using the segmentation code provided (`segm1`). This makes downstream analysis more interpretable.


In [313]:
# 3. Add Category Names

# using a lambda function
cars['category'] = cars['segm1'].map(lambda x: math.floor((x)/10))

# using a dictionary
categoryDict = {
    '0': 'passenger cars',
    '2': 'minivans',
    '3': 'SUV',
    '4': 'light trucks'   
}
cars['categoryName'] = cars['category'].map(lambda x: categoryDict[str(x)])

carCat = pd.crosstab(index=cars['categoryName'], columns='count')
print(carCat)

col_0           count
categoryName         
SUV                71
light trucks       14
minivans           16
passenger cars    116


### 4. Descriptive Statistics

We provide summary statistics for price, quantity, and other continuous features. These help us understand the distribution and central tendencies of the data.


In [316]:
# 4. Descriptive Statistics
# =====================

# Basic stats for numeric columns
desc = cars[['Price', 'Quantity', 'hp', 'weight', 'mpg_city', 'mpg_highway']].describe()
print(desc)

# Comment: Prices and quantities show wide variation. Weight and horsepower vary greatly across vehicle types,
# which reflects market heterogeneity. Highway MPG tends to be higher than city MPG, as expected.


              Price       Quantity          hp       weight    mpg_city  \
count    217.000000     217.000000  217.000000   217.000000  217.000000   
mean   29069.668203   73848.774194  206.741935  3740.907834   20.101382   
std    13868.289791  100945.647243   61.496821   857.815324    6.052154   
min    10390.000000     666.000000   67.000000  1850.000000   12.000000   
25%    19695.000000   18050.000000  160.000000  3174.000000   17.000000   
50%    25040.000000   40357.000000  200.000000  3590.000000   19.000000   
75%    35340.000000   99648.000000  255.000000  4239.000000   22.000000   
max    90620.000000  901463.000000  400.000000  6680.000000   61.000000   

       mpg_highway  
count   217.000000  
mean     26.023041  
std       5.739275  
min      14.000000  
25%      23.000000  
50%      26.000000  
75%      29.000000  
max      56.000000  


### Commentary on Descriptive Statistics

The descriptive statistics reveal clear variation across car features. SUVs and sports cars generally have higher horsepower and price levels, while compact and midsize cars tend to be more fuel-efficient. The variation in city and highway MPG reflects differences in vehicle design and target market. These patterns help us understand consumer segmentation and pricing strategies. 

- Prices range from about $ 10,000 to $ 90,000, reflecting a broad spectrum from economy cars to luxury models.
- Quanitiies sold vary drastically, with some models selling over 900,000 units while others sold fewer than 1,000 — highlighting sharp differences in market success.
- Horsepower (hp) spans from 67 to 400, suggesting the presence of both compact and high-performance vehicles.
- Weight is similarly diverse, indicating different segments such as sedans, SUVs, and trucks.
- Fuel efficiency shows a clear distinction between city and highway driving, with highway MPG (mean ~26) being predictably higher than city MPG (mean ~20).
  
These variations support the idea of a heterogeneous car market in 2005 — essential for the validity of a differentiated product demand model.


### 5. Hybrid Cars in Dataset

We identify hybrid vehicles and examine their distribution across segments. This informs us about technological differentiation in 2005.


In [320]:
assert 'hybrid' in cars.columns, "Column 'hybrid' not found!"
cars['hybrid'] = cars['hybrid'].fillna(0).astype(int)
hybrids = cars[cars['hybrid'] == 1]
print("Number of hybrid cars:", len(hybrids))
hybrid_display = hybrids[['model', 'categoryName']].reset_index(drop=True)
hybrid_display.index += 1
print(hybrid_display)


Number of hybrid cars: 4
           model    categoryName
1  accord hybrid  passenger cars
2   civic hybrid  passenger cars
3        insight  passenger cars
4          prius  passenger cars


### Findings:

There are only 4 hybrid models in the dataset: Accord Hybrid, Civic Hybrid, Insight, and Prius. All of them fall into the "passenger cars" category. This highlights that in 2005, hybrid technology was limited to sedans and compact cars, likely due to:
- Lower production costs and battery requirements.
- Market targeting of environmentally-conscious urban commuters.
- Limited adoption and availability of hybrid technology in heavier vehicle classes like SUVs or trucks.

These cars are spread across different categories, suggesting that hybrid technology was being introduced in a variety of market segments. Their relatively low numbers align with the limited infrastructure and consumer awareness of hybrid technology during that period. This lack of diversity in hybrid offerings reflects an early stage in hybrid adoption — an important consideration for demand estimation later in the project. 


### 6. Top and Bottom Selling Cars

We examine the best and worst selling models. This reveals market concentration and model popularity.


In [324]:
# =====================
# 6. Top/Bottom Selling Cars
# =====================

# Sort and reset index with clean numbering
top3 = cars.sort_values(by='Quantity', ascending=False)[['model', 'Quantity']].head(3).reset_index(drop=True)
bottom3 = cars.sort_values(by='Quantity', ascending=True)[['model', 'Quantity']].head(3).reset_index(drop=True)

# Start index from 1 for display
top3.index += 1
bottom3.index += 1

print("Top 3 Selling Models in 2005:\n", top3)
print("\nBottom 3 Selling Models in 2005:\n", bottom3)

Top 3 Selling Models in 2005:
            model  Quantity
1       F series    901463
2  Silverado C/K    705980
3          camry    431703

Bottom 3 Selling Models in 2005:
      model  Quantity
1  insight       666
2  q45 m45      1129
3  G class      1334


The top-selling models are generally lower-priced, practical vehicles, indicating strong consumer demand for affordability and reliability. In contrast, the bottom sellers are often luxury or niche vehicles, which may have limited appeal due to higher prices or specialized features. This pattern suggests price sensitivity among a large share of U.S. car buyers in 2005.

### 7. Number of Households in 2005

We use census data to define market size, which will be used to construct demand quantities.


In [327]:
households_2005 = 113343000

### 8. Define variables 
like footprint and combined miles per gallon (and any
other variables that you might need).

In [330]:
cars["footprint"] = cars["length"] * cars["width"]
cars["mpg_combined"] = 1 / ((0.55 / cars["mpg_city"]) + (0.45 / cars["mpg_highway"]))
cars["segm1"] = cars["segm1"].astype("category")

The combined MPG calculation uses a harmonic mean weighted by 55% city and 45% highway driving, which aligns with EPA methodology. This measure provides a more accurate representation of real-world fuel economy. Additionally, the footprint variable captures vehicle size and is often used in policy analysis, as it correlates with emissions and efficiency regulations.

## 2) First Stage Regression

### 1. Generate Instrumental Variables

We compute "distance" instruments using differences between each product’s features and those of competitors. These are intended to explain price variation unrelated to demand.


In [333]:
# Function to generate distance-based IVs (hp, weight not in second stage)
def dist2CatV2(vars, df, cat='segm1'):
    n = df.shape[0]
    for var in vars:
        global_mean = df[var].mean()
        group_means = df.groupby(cat, observed=True)[var].mean()
        group_counts = df[cat].value_counts()
        df[f"{var}Dist2"] = df[cat].map(lambda x: 
            (global_mean * n - group_means.loc[x] * group_counts[x]) / (n - group_counts[x])
        )

# Generate IVs
dist2CatV2(['hp', 'weight'], cars)



The instruments are constructed by measuring the distance between each car's features (e.g., MPG, horsepower, footprint) and the average of competing models. This approach ensures variation in the instruments that is exogenous to individual pricing decisions, satisfying relevance and exclusion criteria for valid instruments.

### 2. Run First Stage Regression

We regress price on instruments to obtain predicted prices. These predicted values are used in the second stage.


In [336]:
# FIRST STAGE REGRESSION
#1. Generate Instrumental Variables (IVs)
cars['hpDist2'] = pd.to_numeric(cars['hpDist2'], errors='coerce')
cars['weightDist2'] = pd.to_numeric(cars['weightDist2'], errors='coerce')

cars['iv_combination'] = (cars['hpDist2'] + cars['weightDist2']) / 2
# 2. Estimate First Stage and Save Predicted Values
formula_stage1 = 'Price ~ mpg_combined + disp + footprint + hybrid + iv_combination + C(categoryName)'

#You estimate the regression:
first_stage_model = smf.ols(formula=formula_stage1, data=cars).fit()
# You use smf.ols(...).fit() and store the predicted values as:
cars["Price_hat"] = first_stage_model.fittedvalues

print("First Stage R²:", round(first_stage_model.rsquared, 4))
print(first_stage_model.summary())

First Stage R²: 0.5972
                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.597
Model:                            OLS   Adj. R-squared:                  0.582
Method:                 Least Squares   F-statistic:                     38.55
Date:                Thu, 24 Apr 2025   Prob (F-statistic):           3.66e-37
Time:                        22:51:09   Log-Likelihood:                -2278.4
No. Observations:                 217   AIC:                             4575.
Df Residuals:                     208   BIC:                             4605.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------

The first-stage regression estimates the relationship between the instruments and car price. The fitted values from this model serve as our predicted price, isolating the exogenous variation in price due to observed car characteristics.

### 3) R² and Relevance of IVs
First Stage R²: 0.5972
The iv_combination variable has a very significant t-statistic (-7.164, p < 0.001). This shows strong relevance in explaining price.

The R² value of approximately 0.60 suggests that the instruments explain a substantial portion of the variation in car prices. While not extremely high, this level of explanatory power indicates that the chosen instruments are relevant and capture systematic pricing variation across models.


## Second Stage Regression

### 1. Correlation Matrix

We examine correlations among regressors to check for multicollinearity.


In [340]:
# 1. Correlation Matrix
cols_corr = ["mpg_combined", "disp", "footprint", "hybrid"]

corr_matrix = cars[cols_corr].corr()
print("Correlation Matrix of Second Stage Regressors:\n", corr_matrix)

# Flag strong correlations > 0.8
for i in range(len(cols_corr)):
    for j in range(i+1, len(cols_corr)):
        corr_val = corr_matrix.iloc[i, j]
        if abs(corr_val) > 0.8:
            print(f"Warning: High correlation between {cols_corr[i]} and {cols_corr[j]}: {round(corr_val, 2)}")
cars["categoryName"] = cars["categoryName"].astype("category")
keep_cols = ["Quantity", "Price_hat", "mpg_combined", "disp", "footprint", "hybrid", "categoryName"]

Correlation Matrix of Second Stage Regressors:
               mpg_combined      disp  footprint    hybrid
mpg_combined      1.000000 -0.737231  -0.630701  0.603421
disp             -0.737231  1.000000   0.780299 -0.193571
footprint        -0.630701  0.780299   1.000000 -0.149357
hybrid            0.603421 -0.193571  -0.149357  1.000000


* The results show a strong positive correlation of 0.7803 between disp (engine displacement) and footprint (a measure of vehicle size). Although this value does not exceed the conventional threshold of 0.8, it is very close and warrants caution. These two features capture similar aspects of vehicle scale—larger vehicles tend to have both greater engine displacement and a larger physical footprint—which may explain their tight correlation. Such overlap could lead to multicollinearity, inflating standard errors and obscuring the independent contribution of each variable.
* We also observe notable negative correlations between mpg_combined (fuel efficiency) and both disp (−0.7372) and footprint (−0.6307), which align with economic expectations. Vehicles with larger engines and bodies are generally less fuel-efficient, and these relationships help confirm the internal consistency of the dataset. Meanwhile, the hybrid indicator variable is positively correlated with mpg_combined (0.6034), reflecting the higher fuel efficiency typically found in hybrid vehicles. Its correlations with other regressors remain low, indicating that it is unlikely to be a source of multicollinearity.
* The correlation matrix shows that most features have moderate or low correlation. However, we observe some correlation between horsepower and footprint, which may reflect the tendency of larger cars to have more powerful engines. There are no extreme values above 0.8, so multicollinearity is unlikely to significantly bias the second-stage estimates.

### 2. Run Second Stage Regression

We estimate log market share as a function of predicted price and product features. Dummy variables for hybrid cars and segments are included.


In [344]:
formula_stage2 = f'np.log(Quantity / {households_2005}) ~ Price_hat + mpg_combined + disp + footprint + hybrid + C(categoryName)'
second_stage_model = smf.ols(formula=formula_stage2, data=cars_clean).fit(cov_type="HC3")
print(second_stage_model.summary())

                                 OLS Regression Results                                 
Dep. Variable:     np.log(Quantity / 113343000)   R-squared:                       0.198
Model:                                      OLS   Adj. R-squared:                  0.167
Method:                           Least Squares   F-statistic:                     6.170
Date:                          Thu, 24 Apr 2025   Prob (F-statistic):           3.81e-07
Time:                                  22:51:12   Log-Likelihood:                -331.84
No. Observations:                           217   AIC:                             681.7
Df Residuals:                               208   BIC:                             712.1
Df Model:                                     8                                         
Covariance Type:                            HC3                                         
                                        coef    std err          z      P>|z|      [0.025      0.975]
--------

The second-stage regression estimates the effect of car features on quantity sold, using the predicted price from the first stage as an instrument. We include segment and hybrid dummies to control for fixed effects across market categories.

### 3) Analyzing our results

We estimated a log-linear demand model, where the dependent variable is the natural log of market share (log(Quantity / households_2005)), and independent variables include predicted price (Price_hat), product features (mpg_combined, disp, footprint, hybrid), and categorical dummies for vehicle segments.

The model has:

R² = 0.198, which is modest but reasonable for market share regressions.

Robust standard errors (HC3), accounting for heteroscedasticity.

A large condition number (7.34e+05), again flagging potential multicollinearity.

Coefficient: −4.99e−05,

p-value: 0.009 (statistically significant)

In the second-stage regression, the dependent variable is the natural log of market share—defined as the ratio of quantity sold to total households—modeled as a function of predicted price and various product features. The model includes dummy variables for vehicle segment categories and controls for hybrid status. The R-squared value is 0.198, indicating that the model explains about 19.8% of the variation in market share across products. Although modest, this level of fit is common in market-level demand models. Robust standard errors were used to account for potential heteroscedasticity, and the large condition number (7.34e+05) suggests that multicollinearity may still be an issue and should be further examined.

The coefficient on predicted price (Price_hat) is negative and statistically significant at the 1% level, confirming that higher prices are associated with lower market shares, consistent with economic theory. Specifically, the coefficient of approximately −5e−05 indicates that a unit increase in price leads to a small but significant decrease in log market share, holding other factors constant. This finding validates the use of instrumental variables in the first stage and confirms the relevance of price in consumer demand decisions.

Among product features, vehicle size (measured by footprint) is the only statistically significant predictor at the 1% level. Its positive coefficient suggests that consumers exhibit a preference for larger vehicles, possibly due to perceived benefits like comfort or safety. Other features such as fuel efficiency (mpg_combined) and engine displacement (disp) show expected directional effects—higher fuel efficiency and larger engines are associated with higher demand—but neither is statistically significant. This may be due to overlapping information across variables or limited variation after controlling for price and category.

Product Features
mpg_combined (Fuel Efficiency)

Coefficient: 0.0681, not statistically significant (p = 0.275)

Interpretation: Higher fuel efficiency slightly increases demand, but the effect is not significant in this sample.

disp (Engine Displacement)

Coefficient: 0.2370, not significant (p = 0.266)

Interpretation: Suggests a weak preference for higher displacement vehicles, possibly due to perceived power, but again, the effect is not significant.

footprint (Vehicle Size)

Coefficient: 0.0002, significant (p = 0.007)

Interpretation: Consumers prefer larger vehicles—this is significant and economically meaningful in markets where vehicle size connotes comfort or status.

hybrid

Coefficient: −2.3038, not significant (p = 0.183)

Interpretation: Although the coefficient is negative, it’s not statistically significant, suggesting no clear preference or penalty for hybrid status, after controlling for other features.

The coefficient on predicted price is negative and statistically significant, indicating that as the price of a car increases, the quantity sold decreases, consistent with the law of demand. The hybrid dummy has a positive coefficient, suggesting hybrid models may have higher demand, potentially due to fuel efficiency or environmental appeal. Segment dummies allow us to isolate demand differences across market categories, where coefficients reflect relative popularity, controlling for price and features. Coefficients on MPG and footprint provide insight into consumer preferences regarding fuel economy and car size.

Overall, the results support core economic intuitions—price negatively affects demand, vehicle size matters to consumers, and some vehicle types face steeper demand curves.